### Wczytanie Bibliotek

In [ ]:
from PIL import Image
import sys
import numpy as np
import cv2
import tifffile
import plotly.express as px
from IPython import display
import pandas as pd
from timeit import timeit
from memory_profiler import profile
import random
from IPython.display import IFrame
import ipywidgets as widgets
import plotly
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances as pairwise_dist
import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
from skimage import exposure, img_as_ubyte
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from ipywidgets import fixed
from memory_profiler import profile
%load_ext memory_profiler

### Wczytanie plików Experiment-392

In [ ]:
file_red = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=r.tif'
file_green = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=g.tif'
file_blue = 'dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=b.tif'

tif_red = tifffile.imread(file_red)
tif_green = tifffile.imread(file_green)
tif_blue = tifffile.imread(file_blue)

### Normalizacja plików exp-392

In [ ]:
# przykształcenie tifów 16-bitowych na 8-bitowe

# inicjalizacja pustych tifów 8 bitowych
tif_red_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')
tif_green_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')
tif_blue_norm = np.empty(shape=(18, 1040,1388), dtype='uint8')

# normalizacja
for i in range(0,tif_blue.shape[0]):
    tif_red_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_red[i]))
    tif_green_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_green[i]))
    tif_blue_norm[i] = img_as_ubyte(exposure.rescale_intensity(tif_blue[i]))  

In [ ]:
def get_coords(tif_arr):

    z, y, x = tif_arr.shape[:3]

    x_size = x
    y_size = y
    z_size = z

    z_list = []
    for i in range(z_size):
        z_list += [i] * (y_size * x_size)

    y_list = []
    for i in range(y_size):
        y_list += [i] * (x_size)
    y_list *= z_size

    x_list = []
    x_list = np.arange(0, x_size).tolist() * (z_size * y_size)
    
    return np.c_[z_list, y_list, x_list]

### połączenie współrzędnych, kolorów i rozmiarów pixeli pliku exp-392 w jeden array 2d

In [ ]:
tif_red_flat = tif_red_norm.flatten()
tif_green_flat = tif_green_norm.flatten()
tif_blue_flat = tif_blue_norm.flatten()

coords=get_coords(tif_red_norm)

tif_arr = np.c_[coords, tif_red_flat, tif_green_flat, tif_blue_flat, np.ones((tif_red_norm.size),dtype='int32')]

### Wczytanie kolorowego pliku Image0026-rgb

In [ ]:
img26_file = 'dataset/rgb/FISH_Brachy/Image0026-rgb.tif' 

# wczytanie przy pomocy biblioteki tifffile (OpenCV nie odnajdywało warst)
img26 = tifffile.imread(img26_file)

### Wczytanie czarno-białego pliku focal1.tiff

In [ ]:
# pobranie lokalnej ścieżki do pliku

f1_file = 'dataset/EmbryoCE/focal1.tif'

# wczytanie pliku przy pomocy biblioteki tifffile (openCV nie wczytywał trzeciego wymiaru)
f1 = tifffile.imread(f1_file)

# odwrócenie skaliszarości o 180 stopnii ( ponieważ po wczytaniu została ona zmieniona )
f1 = cv2.bitwise_not(f1)

### połączenie współrzędnych, kolorów i rozmiarów pixeli pliku img26 w jeden array 2d

In [ ]:
img26_flat = img26.flatten()
coords=get_coords(img26)

img26_arr = np.c_[coords, img26_flat[::3], img26_flat[1::3], 
                img26_flat[2::3], np.ones((img26_flat[::3].size),dtype=int)]

### Usuwanie czarnego tła

In [ ]:
def clear_black_bg(tif_arr, threshold):
    tif_arr = tif_arr[(tif_arr[:,3]>threshold)
                      |(tif_arr[:,4]>threshold)
                      |(tif_arr[:,5]>threshold)]
    return tif_arr

In [ ]:
img26_cl=clear_black_bg(img26_arr,100)

In [ ]:
tif_arr_cl=clear_black_bg(tif_arr,100)

### funkcje pomocnicze

In [ ]:
def get_centroid(arr):
    length = arr.shape[0]
    sum_z = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_x = np.sum(arr[:, 2])
    sum_r = np.sum(arr[:, 3])
    sum_g = np.sum(arr[:, 4])
    sum_b = np.sum(arr[:, 5])
    return sum_z/length, sum_y/length, sum_x/length, sum_r/length, sum_g/length, sum_b/length, length

In [ ]:
def get_medoid(arr):
    distMatrix=pairwise_dist(arr[:,:6])
    medoid=arr[[np.argmin(distMatrix.sum(axis=0))]]
        
    medoid[0,6]=arr.shape[0]
    
    return medoid

In [ ]:
def get_neighbours(arr, pixel, radius, zradius, color_diff):
    neighbours = arr[((arr[:,0]<=pixel[0]+zradius) 
                            & (arr[:,0]>=pixel[0]-zradius)
                            & (arr[:,1]<=pixel[1]+radius)
                            & (arr[:,1]>=pixel[1]-radius)
                            & (arr[:,2]<=pixel[2]+radius)
                            & (arr[:,2]>=pixel[2]-radius)
                            & (arr[:,3]<=pixel[3]+color_diff)
                            & (arr[:,3]>=pixel[3]-color_diff)
                            & (arr[:,4]<=pixel[4]+color_diff)
                            & (arr[:,4]>=pixel[4]-color_diff)
                            & (arr[:,5]<=pixel[5]+color_diff)
                            & (arr[:,5]>=pixel[5]-color_diff))]
    return neighbours

In [ ]:
def get_neighbours2(arr, pixel, radius, zradius):
    neighbours = arr[((arr[:,0]<=pixel[0]+zradius) 
                            & (arr[:,0]>=pixel[0]-zradius)
                            & (arr[:,1]<=pixel[1]+radius)
                            & (arr[:,1]>=pixel[1]-radius)
                            & (arr[:,2]<=pixel[2]+radius)
                            & (arr[:,2]>=pixel[2]-radius))]
    return neighbours

In [ ]:
def get_neighbours_by_color(arr, pixel, color_diff):
    neighbours = arr[((arr[:,3]<=pixel[3]+color_diff)
                            & (arr[:,3]>=pixel[3]-color_diff)
                            & (arr[:,4]<=pixel[4]+color_diff)
                            & (arr[:,4]>=pixel[4]-color_diff)
                            & (arr[:,5]<=pixel[5]+color_diff)
                            & (arr[:,5]>=pixel[5]-color_diff))]
    return neighbours

In [ ]:
def cap_data(arr):
    for i in range(3,6):
        print(i)
        p1 = np.quantile(n[:,i], 0.1)
        p2 = np.quantile(n[:,i], 0.9)
        print(p1)
        print(p2)
        
        arr=arr[((arr[:,i]>=p1) & (arr[:,i]<=p2))]

    return arr

### Metoda I - centroid + róznica koloru + bez usuwania

In [ ]:
def centroid_method(tif_arr, radius, zradius, color_diff):
    
    centroids = np.empty([0,7])
    
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours(tif_arr, pixel, radius, zradius, color_diff)

        # calculating centroid vector from array of neighbours
        centroid = get_centroid(neighbours_arr)

        # adding centroid to centroids matrix
        centroids=np.vstack([centroids, centroid])

        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
        
    return centroids

### Metoda II - centroid + róznica koloru + z usuwaniem

In [ ]:
def centroid_method2(tif_arr, radius, zradius, color_diff):
    
    centroids = np.empty([0,7])
    
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
        
        neighbours_arr = get_neighbours_by_color(neighbours_arr, pixel, color_diff)

        # calculating centroid vector from array of neighbours
        centroid = get_centroid(neighbours_arr)

        # adding centroid to centroids matrix
        centroids=np.vstack([centroids, centroid])
        
    return centroids

### Metoda III - centroid + bez różnicy koloru + nie ma outlierów

In [ ]:
def centroid_method3(tif_arr, radius, zradius):
    
    centroids = np.empty([0,7])
    
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)

        # calculating centroid vector from array of neighbours
        centroid = get_centroid(neighbours_arr)

        # adding centroid to centroids matrix
        centroids=np.vstack([centroids, centroid])

        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
        
    return centroids

### Metoda I - medoid + róznica koloru + bez usuwania

In [ ]:
def medoid_method(tif_arr,radius, zradius, color_diff):
    medoids = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
        
        medoid=get_medoid(neighbours_arr)
        
        medoids=np.vstack([medoids, medoid])
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
    
    return medoids

### Metoda II - medoid + róznica koloru + z usuwaniem

In [ ]:
def medoid_method2(tif_arr,radius, zradius, color_diff):
    medoids = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
        
        neighbours_arr=get_neighbours_by_color(neighbours_arr, pixel, color_diff)

        medoid=get_medoid(neighbours_arr)
        
        medoids=np.vstack([medoids, medoid])
    
    return medoids

### Metoda III - medoid + bez różnicy koloru + nie ma outlierów

In [ ]:
def medoid_method3(tif_arr,radius, zradius):
    medoids = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]

        medoid=get_medoid(neighbours_arr)
        
        medoids=np.vstack([medoids, medoid])
    
    return medoids

### Metoda I - mediana + róznica koloru + bez usuwania

In [ ]:
def median_method(tif_arr,radius, zradius, color_diff):
    medians = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
        
        median=np.median(neighbours_arr,axis=0,keepdims=True)
        median[0,6]=neighbours_arr.shape[0]
        
        medians=np.vstack([medians, median])
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
    
    return medians

### Metoda II - mediana + róznica koloru + z usuwaniem

In [ ]:
def median_method2(tif_arr,radius, zradius, color_diff):
    medians = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]
        
        neighbours_arr=get_neighbours_by_color(neighbours_arr, pixel, color_diff)

        median=np.median(neighbours_arr,axis=0,keepdims=True)
        median[0,6]=neighbours_arr.shape[0]
        
        medians=np.vstack([medians, median])
    
    return medians

### Metoda III - mediana + bez różnicy koloru + nie ma outlierów

In [ ]:
def median_method3(tif_arr,radius, zradius):
    medians = np.empty([0,7])
    # looping thorugh array until it's empty
    while len(tif_arr)>0:

        # assign starting point to random pixel         
        pixel = random.choice(tif_arr)

        # creating matrix of neighbours
        neighbours_arr = get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # deleting neighbours from tif_arrlicated array
        dims = np.maximum(neighbours_arr.max(0),tif_arr.max(0))+1
        tif_arr = tif_arr[~np.in1d(np.ravel_multi_index(tif_arr.T,dims),np.ravel_multi_index(neighbours_arr.T,dims))]

        median=np.median(neighbours_arr,axis=0,keepdims=True)
        median[0,6]=neighbours_arr.shape[0]
        
        medians=np.vstack([medians, median])
    
    return medians

### Vizualizacja w plotly graph object

In [ ]:
def plotly_vis(arr, size_min, size_max):
    colors_arr = arr[:,3:6]
    colors_arr = [f'rgb({"{},{},{}".format(color[0],color[1],color[2])})' for color in colors_arr]
    trace1 = go.Scatter3d(
        x=arr[:,2], 
        y=arr[:,1], 
        z=arr[:,0],
        mode="markers",
        marker_color=colors_arr,
        text=colors_arr,
        customdata=arr[:,6],
        marker=dict(
            sizemode="area",
            sizeref=arr[:,6].max() / size_max ** 2,
            sizemin=size_min,
            opacity=1,
            size=arr[:,6],
            line=dict(width=0)
        ),
        hovertemplate='<br>x:%{x}<br>y:%{y}<br>z:%{z}<br>color: %{text}<br>size: %{customdata}'
    )

    data = [trace1]
    title = "Tiff 3D"
    layout = go.Layout(height=800, width=1000, title=title)

    fig = go.Figure(data=data, layout=layout)

    fig.show()     

#### Widgety i interakcja

In [ ]:
@widgets.interact_manual(method=['centroid_method','centroid_method2','centroid_method3',
                                 'medoid_method','medoid_method2','medoid_method3',
                                 'median_method','median_method2','median_method3'],
                        tiff=fixed(img26_cl), 
                        radius=widgets.IntSlider(value=5, min=1, max=20, continuous_update=False), 
                        zradius=widgets.IntSlider(value=2, min=1, max=5, continuous_update=False), 
                        color_diff=widgets.IntSlider(value=10, min=1, max=40, continuous_update=False)
                        ).options(manual_name="update")
def tif_reduction_widget(method,tiff, radius, zradius, color_diff):
    
    if method == 'centroid_method':                
        data_points = centroid_method(tiff,radius,zradius,color_diff)
        
    elif method == 'centroid_method2':
        data_points = centroid_method2(tiff,radius,zradius,color_diff)
        
    elif method == 'centroid_method3':
        data_points = centroid_method3(tiff,radius,zradius)
                
    elif method == 'medoid_method':
        data_points = medoid_method(tiff,radius,zradius,color_diff)
                
    elif method == 'medoid_method2':
        data_points = medoid_method2(tiff,radius,zradius,color_diff)
                
    elif method == 'medoid_method3':
        data_points = medoid_method3(tiff,radius,zradius)
        
    elif method == 'median_method':
        data_points = median_method(tiff,radius,zradius,color_diff)
                
    elif method == 'median_method2':
        data_points = median_method2(tiff,radius,zradius,color_diff)
                
    elif method == 'median_method3':
        data_points = median_method3(tiff,radius,zradius)
        
    

    print(data_points.shape)
    widgets.interact(plotly_vis, arr=fixed(data_points),
                     size_min=widgets.IntSlider(value=10, min=0, max=20, continuous_update=False), 
                     size_max=widgets.IntSlider(value=50, min=40, max=65, continuous_update=False))

### Pomiary

In [ ]:
%timeit centroid_method(img26_cl,3,1,5)

In [ ]:
%timeit centroid_method2(img26_cl,3,1,5)

In [ ]:
%timeit centroid_method3(img26_cl,3,1,5)

In [ ]:
%timeit medoid_method(img26_cl,3,1,5)

In [ ]:
%timeit medoid_method2(img26_cl,3,1,5)

In [ ]:
%timeit medoid_method3(img26_cl,3,1,5)

In [ ]:
%timeit median_method(img26_cl,3,1,5)

In [ ]:
%timeit median_method2(img26_cl,3,1,5)

In [ ]:
%memit centroid_method3(img26_cl,3,1,5)

In [ ]:
%memit medoid_method(img26_cl,5,1,10)

In [ ]:
# test dataset
b = np.random.randint(0, 255, [10, 7]).astype('float64')
b

In [ ]:
import cProfile
def build():
    arr = []
    for a in range(0, 1000000):
        if a % 2 == 0:
            arr.append(a)
if __name__ == '__main__':
    cProfile.run('build()')

### Brudnopis

In [ ]:
voxel=random.choice(img26_cl)
voxel

In [ ]:
n=get_neighbours(img26_cl,voxel,5,1,5)

In [ ]:
n2=get_neighbours(img26_cl,voxel,5,1,300)
n2

In [ ]:
n2.shape

In [ ]:
medoid=get_medoid(n)

In [ ]:
medoid

In [ ]:
medoid2=get_medoid(n2)
medoid2

In [ ]:
n3=get_neighbours(img26_cl,voxel,5,1,10)
print(n3)
medoid3=get_medoid(n3)
medoid3

In [ ]:
distMatrix=pairwise_dist(n[:,:6])
distMatrix

In [ ]:
# medoid=arr[[np.argmin(distMatrix.sum(axis=0))]]
distMatrix.sum(axis=0)

In [ ]:
n[[np.argmin(distMatrix.sum(axis=0))]]

In [ ]:
median=np.median(n,axis=0,keepdims=True)

In [ ]:
median

In [ ]:
median[0,6]=n.shape[0]

In [ ]:
n.shape[0]

In [ ]:
median

In [ ]:
m=medoid_method(img26_cl,5,1,10)

In [ ]:
m.shape